# Enfoque del Análisis por Dimensión - Ámbito 1: Aspectos de Estructura Textual

En este documento se sigue el enfoque de **probar la modalidad de los prompts por dimensión** en el ámbito 1 de **Aspectos de estructura textual**, considerando específicamente las dimensiones de:

- Ortografía  
- Vocabulario  
- Cohesión  

Para hacer los resultados más **entendibles y analizables**, se evalúan los 20 textos enviados por Nicolás y por mi para tener un marco de comparación para los resultados que entreguen las consultas a la API de Azure OpenIA.  

Se ha decidido **descartar el uso de `LangChain`**, debido a que no se ajusta a los objetivos de este análisis y podría introducir complejidades innecesarias en la evaluación de las dimensiones textuales.  

A continuación, se muestra cómo **el prompt es enviado al texto**, así como el **prompt completo**, que abarca de manera ambiciosa **todas las dimensiones del Ámbito 1**, según el detalle que se puede observar en el contenido del prompt.


### Ortografía

In [ ]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_ortografia = """
Eres un asistente experto en corrección de textos. Aplica estas reglas al texto:

ORTOGRAFÍA
   - Ortografía acentual: errores en tildes según normas generales y especiales.
       Ejemplos: esta/está, animos/ánimos
   - Ortografía literal: errores en letras dentro de palabras.
       Omisión: "alchol" → "alcohol"
       Sustitución: "fotocopeo" → "fotocopio"
       Adición: "rrallado" → "rallado"
       Ejemplos adicionales: "apartir" → "a partir", "através" → "a través", "porsiacaso" → "por si acaso"
   - Ortografía puntual: errores en signos de puntuación según normas RAE.
       Incluye: coma, punto, punto y coma, signos de exclamación e interrogación

INSTRUCCIONES CRÍTICAS:
- Detecta errores según estas categorías y subcategorías.
- NO incluyas el texto corregido completo.
- Devuelve SOLAMENTE JSON válido sin formato markdown (sin ```json o similares).
- El JSON debe comenzar inmediatamente con {{ y terminar con }}.

ESTRUCTURA EXACTA REQUERIDA:
{
  "texto_original_preview": "Primeros 100 caracteres del texto...",
  "errores_detectados": {
        "ortografia acentual": ["error → corrección"],
        "ortografia literal": ["error → corrección"],
        "ortografia puntual": ["error → corrección"]
  }
}

- En "texto_original_preview" incluye SOLO los primeros 100 caracteres del texto original.
- En "errores_detectados" lista todos los errores encontrados.
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.
- Solo analiza la categoría de ORTOGRAFÍA, pero mira todo el contexto para detectar errores correctamente.
"""

resultados = []

for i, texto in enumerate(textos, start=1):
    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": prompt_ortografia},
        {"role": "user", "content": texto}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0, (No es soportado en Azure OpenAI GPT5)
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "01")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "ort_macro.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\01\ort_macro.json


### Vocabulario

In [2]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_vocabulario = """
Analiza el siguiente texto en base a la dimensión de VOCABULARIO:

VOCABULARIO
   - Amplitud: evitar repetición innecesaria de sustantivos, adjetivos y adverbios.
       Ejemplos: contaminación/ ecosistema / polución / ambiente
   - Precisión léxica: uso incorrecto de palabras que confunden el sentido; incluye redundancias.
       Ejemplos: “absceso” → “acceso”, “actitud” → “aptitud”; “salir para afuera”, “hace un tiempo atrás”
   - Registro formal: evitar expresiones informales, abreviaciones o coloquialismos.
       Ejemplos: “profe” → “profesor/a”, “finde” → “fin de semana”; “cachar” → “comprender”}

INSTRUCCIONES CRÍTICAS:
- Detecta errores según estas categorías y subcategorías.
- NO incluyas el texto corregido completo.
- Devuelve SOLAMENTE JSON válido sin formato markdown (sin ```json o similares).
- El JSON debe comenzar inmediatamente con {{ y terminar con }}.

ESTRUCTURA EXACTA REQUERIDA:
{
  "texto_original_preview": "Primeros 100 caracteres del texto...",
  "errores_detectados": {
        "vocabulario amplitud": ["error → corrección"],
        "vocabulario precision lexica": ["error → corrección"],
        "vocabulario registro formal": ["error → corrección"]
  }
}

- En "texto_original_preview" incluye SOLO los primeros 100 caracteres del texto original.
- En "errores_detectados" lista todos los errores encontrados.
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.
- Solo analiza la categoría de VOCABULARIO, pero mira todo el contexto para detectar errores correctamente.
"""

resultados = []

for i, texto in enumerate(textos, start=1):
    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": prompt_vocabulario},
        {"role": "user", "content": texto}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0, (No es soportado en Azure OpenAI GPT5)
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "01")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "voc_macro.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\01\voc_macro.json


### Cohesión

In [3]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_cohesion = """
Analiza el siguiente texto en base a la dimensión de COHESIÓN TEXTUAL:

COHESIÓN TEXTUAL
   - Concordancia gramatical: errores de concordancia entre sustantivo/adjetivo, sujeto/predicado, pronombres y consistencia de tiempos verbales.
   - Conexión textual: uso adecuado de preposiciones, locuciones y conectores; evitar queísmo/dequeísmo.
   - Referencias y correferencias: pronombres u otros mecanismos sin referencia clara.
       Ejemplos: “esto”, “lo anterior” sin antecedente claro

INSTRUCCIONES CRÍTICAS:
- Detecta errores según estas categorías y subcategorías.
- NO incluyas el texto corregido completo.
- Devuelve SOLAMENTE JSON válido sin formato markdown (sin ```json o similares).
- El JSON debe comenzar inmediatamente con {{ y terminar con }}.

ESTRUCTURA EXACTA REQUERIDA:
{
  "texto_original_preview": "Primeros 100 caracteres del texto...",
  "errores_cohesion_textual": {
        "concordancia gramatical": ["error → corrección"],
        "conexion textual": ["error → corrección"],
        "referencias y correferencias": ["error → corrección"]
  }
}

- En "texto_original_preview" incluye SOLO los primeros 100 caracteres del texto original.
- En "errores_cohesion_textual" lista todos los errores encontrados.
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.
- Solo analiza la categoría de COHESIÓN TEXTUAL, pero mira todo el contexto para detectar errores correctamente.
"""

resultados = []

for i, texto in enumerate(textos, start=1):
    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": prompt_cohesion},
        {"role": "user", "content": texto}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0, (No es soportado en Azure OpenAI GPT5)
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "01")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "coh_macro.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\01\coh_macro.json
